In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
from utils import load_config
options = load_config("./config.json")

In [3]:
from loadDataset import LoadDataset
dataset = LoadDataset(options)

Loading all datasets...
train dataset shape: (940, 3)
train dataset family number: 47
test dataset shape: (200, 3)
test dataset family number: 10
val dataset shape: (200, 3)
val dataset family number: 10


In [4]:
from fcgVectorize import FCGVectorize

vectorizer = FCGVectorize(options, dataset)
# node embedding for raw data
vectorizer.node_embedding(dataset.rawDataset)

Word2vec model exist, load word2vec model...
Start to get node embedding...


100%|██████████| 1340/1340 [00:00<00:00, 27550.82it/s]

Finish getting node embedding


In [5]:
from train import TrainModule
trainModule = TrainModule(options, dataset)

In [6]:
from tqdm import tqdm
import torch
import pickle
from torch_geometric.utils.convert import from_networkx
from sklearn import preprocessing as labelEncoder

labels = []
graphList = []

for index, row in tqdm(trainModule.trainDataset.iterrows(), total=trainModule.trainDataset.shape[0]):
    cpu = row["CPU"]
    family = row["family"]
    fileName = row["file_name"]
    filePath = f"{trainModule.embeddingFolder}/{cpu}/{family}/{fileName}.gpickle"
    with open(filePath, "rb") as f:
        fcg = pickle.load(f)
    for node in fcg.nodes:
        fcg.nodes[node]["x"] = torch.nan_to_num(fcg.nodes[node]["x"], nan=0.0)
    torch_data = from_networkx(fcg)
    labels.append(family)
    graphList.append(torch_data)
le = labelEncoder.LabelEncoder()
le.fit(labels)
labels_ = le.transform(labels)
for i, data in enumerate(graphList):
    data.y = torch.tensor(labels_[i])
    data.num_nodes = len(data.x)



  0%|          | 0/940 [00:00<?, ?it/s]

TypeError: nan_to_num(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [10]:
from dataset import FcgSampler
sampler = FcgSampler(labels_, trainModule.support_shots, trainModule.class_per_iter, trainModule.iterations)

In [11]:
from torch_geometric.data import Batch
def collate_graphs(batch):
    return Batch.from_data_list(batch)

In [12]:
from torch_geometric.loader import DataLoader
import numpy as np
loader = DataLoader(graphList, batch_sampler=sampler, num_workers=32, collate_fn=collate_graphs) 

In [18]:
loader2 = DataLoader(graphList, batch_size=50, num_workers=32)

In [15]:
print(next(iter(loader)))

DataBatch(x=[100], edge_index=[2, 987371], y=[100], num_nodes=256437, batch=[256437], ptr=[101])


In [21]:
print("data.x shape:", graphList[0].x.shape)
print("data.x type:", type(graphList[0].x))

AttributeError: 'list' object has no attribute 'shape'